In [1]:

using Random
using CSV
using MLJ
using DataFrames
import DataFramesMeta as DFM
using DelimitedFiles
using MLJScikitLearnInterface
#df = DataFrame(CSV.File(ARGS[1]))
#df = DataFrame("../../features.csv")
using Logging

#logger = ConsoleLogger(stderr, Logging.Debug)
#logger = ConsoleLogger(stderr, Logging.Info)
#logger = ConsoleLogger(stderr, Logging.LogLevel(5000))
#global_logger(logger)


In [2]:
features_csv = "../../data/features.csv"
data, header = readdlm(features_csv, ',', header=true)
df = DataFrame(data, vec(header))
for col in names(df)
    println(col)
end

CountSettlement
CountCity
CountRoad
SumWoodDiceWeight
SumBrickDiceWeight
SumPastureDiceWeight
SumStoneDiceWeight
SumGrainDiceWeight
CountPortWood
CountPortBrick
CountPortPasture
CountPortStone
CountPortGrain
CountHandWood
CountHandBrick
CountHandPasture
CountHandStone
CountHandGrain
CountTotalWood
CountTotalBrick
CountTotalPasture
CountTotalStone
CountTotalGrain
CountDevCardsKnight
CountDevCardsMonopoly
CountDevCardsYearOfPlenty
CountDevCardsRoadBuilding
CountDevCardsVictoryPoint
HasLargestArmy
HasLongestRoad
CountVictoryPoint
HasMostPoints
WonGame


In [3]:
select!(df, Not([
    :CountHandWood,
    :CountHandBrick,
    :CountHandPasture,
    :CountHandStone,
    :CountHandGrain,
    :HasMostPoints,
    :CountVictoryPoint
    ]))

Row,CountSettlement,CountCity,CountRoad,SumWoodDiceWeight,SumBrickDiceWeight,SumPastureDiceWeight,SumStoneDiceWeight,SumGrainDiceWeight,CountPortWood,CountPortBrick,CountPortPasture,CountPortStone,CountPortGrain,CountTotalWood,CountTotalBrick,CountTotalPasture,CountTotalStone,CountTotalGrain,CountDevCardsKnight,CountDevCardsMonopoly,CountDevCardsYearOfPlenty,CountDevCardsRoadBuilding,CountDevCardsVictoryPoint,HasLargestArmy,HasLongestRoad,WonGame
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,3.0,0.0,14.0,7.0,13.0,9.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,14.0,2.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,5.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,4.0,7.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,3.0,14.0,8.0,4.0,7.0,10.0,12.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,5.0,13.0,13.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0
4,2.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,6.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.0,0.0,14.0,7.0,0.0,9.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,17.0,5.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.0,0.0,14.0,7.0,8.0,0.0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,25.0,25.0,3.0,8.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2.0,0.0,6.0,0.0,5.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0,9.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2.0,0.0,13.0,4.0,4.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,17.0,6.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,1.0,1.0,14.0,4.0,4.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,3.0,24.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:


coerce!(df, :WonGame => Multiclass{2})
df = DFM.@transform(df, :WonGame)
df, df_test = partition(df, 0.1, rng=123)

y, X = unpack(df, ==(:WonGame));
y_test, X_test = unpack(df_test, ==(:WonGame));

In [5]:
function load_tree_model()
    @load RandomForestClassifier pkg=BetaML verbosity=0
end

Tree = load_tree_model()
tree = Base.invokelatest(Tree,
    max_depth = 6,
    min_gain = 0.0,
    min_records = 2,
    max_features = 0,
    splitting_criterion = BetaML.Utils.gini)

RandomForestClassifier(
  n_trees = 30, 
  max_depth = 6, 
  min_gain = 0.0, 
  min_records = 2, 
  max_features = 0, 
  splitting_criterion = BetaML.Utils.gini, 
  β = 0.0, 
  rng = Random._GLOBAL_RNG())

In [6]:
mach = machine(tree, X, y)
Base.invokelatest(fit!, mach)


┌ Info: Training machine(RandomForestClassifier(n_trees = 30, …), …).
└ @ MLJBase /home/bryan/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


trained Machine; caches model-specific representations of data
  model: RandomForestClassifier(n_trees = 30, …)
  args: 
    1:	Source @815 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @095 ⏎ AbstractVector{Multiclass{2}}


In [7]:
function analyze_acc(mach, X, y)
    println(typeof(mach), typeof(X))
    p = predict(mach, X)
    yhat = mode.(p)
    acc = accuracy(yhat, y)
    return acc
end

acc = analyze_acc(mach, X, y)
test_acc = analyze_acc(mach, X_test, y_test)
println("acc / test_acc: $acc / $test_acc")

Machine{BetaML.Bmlj.RandomForestClassifier, BetaML.Bmlj.RandomForestClassifier, true}DataFrame
Machine{BetaML.Bmlj.RandomForestClassifier, BetaML.Bmlj.RandomForestClassifier, true}DataFrame
acc / test_acc: 0.945048693149493 / 0.9440674373724115


In [8]:
p = predict(mach, X_test)

268931-element CategoricalDistributions.UnivariateFiniteVector{Multiclass{2}, Float64, UInt32, Float64}:
 UnivariateFinite{Multiclass{2}}(0.0=>0.641, 1.0=>0.359)
 UnivariateFinite{Multiclass{2}}(0.0=>0.99, 1.0=>0.00989)
 UnivariateFinite{Multiclass{2}}(0.0=>0.311, 1.0=>0.689)
 UnivariateFinite{Multiclass{2}}(0.0=>0.75, 1.0=>0.25)
 UnivariateFinite{Multiclass{2}}(0.0=>0.309, 1.0=>0.691)
 UnivariateFinite{Multiclass{2}}(0.0=>0.998, 1.0=>0.00242)
 UnivariateFinite{Multiclass{2}}(0.0=>0.251, 1.0=>0.749)
 UnivariateFinite{Multiclass{2}}(0.0=>0.994, 1.0=>0.00612)
 UnivariateFinite{Multiclass{2}}(0.0=>0.222, 1.0=>0.778)
 UnivariateFinite{Multiclass{2}}(0.0=>0.973, 1.0=>0.0265)
 ⋮
 UnivariateFinite{Multiclass{2}}(0.0=>0.851, 1.0=>0.149)
 UnivariateFinite{Multiclass{2}}(0.0=>0.997, 1.0=>0.00295)
 UnivariateFinite{Multiclass{2}}(0.0=>0.998, 1.0=>0.00234)
 UnivariateFinite{Multiclass{2}}(0.0=>0.752, 1.0=>0.248)
 UnivariateFinite{Multiclass{2}}(0.0=>0.998, 1.0=>0.00234)
 UnivariateFinite{Multiclas

In [ ]:
p[1].

UnivariateFinite{Multiclass{2}}(0.0=>0.641, 1.0=>0.359)

In [18]:
collect(p)[1]

UnivariateFinite{Multiclass{2}}(0.0=>0.641, 1.0=>0.359)

In [22]:
pdf(p, 0.0)

MethodError: MethodError: no method matching pdf(::CategoricalDistributions.UnivariateFiniteVector{Multiclass{2}, Float64, UInt32, Float64}, ::Float64)

Closest candidates are:
  pdf(!Matched::Distributions.Kolmogorov, ::Real)
   @ Distributions ~/.julia/packages/Distributions/tQhJE/src/univariate/continuous/kolmogorov.jl:80
  pdf(!Matched::Distributions.DiscreteUniform, ::Real)
   @ Distributions ~/.julia/packages/Distributions/tQhJE/src/univariate/discrete/discreteuniform.jl:73
  pdf(!Matched::Distributions.Hypergeometric, ::Real)
   @ Distributions ~/.julia/packages/Distributions/tQhJE/src/univariates.jl:644
  ...


In [28]:
v = coerce([0.0, 1.0], Multiclass)
levels(v)

2-element Vector{Float64}:
 0.0
 1.0

In [33]:
M = pdf(p, levels(v))
hcat(M, y_test)

268931×3 Matrix{Float64}:
 0.640547  0.359453    0.0
 0.990115  0.00988541  0.0
 0.310886  0.689114    1.0
 0.749601  0.250399    0.0
 0.309336  0.690664    1.0
 0.997583  0.00241674  0.0
 0.2511    0.7489      1.0
 0.993878  0.00612214  0.0
 0.222223  0.777777    1.0
 0.973467  0.0265332   0.0
 ⋮                     
 0.851408  0.148592    0.0
 0.997052  0.00294837  0.0
 0.997655  0.00234465  0.0
 0.751906  0.248094    0.0
 0.997655  0.00234465  0.0
 0.965991  0.0340089   0.0
 0.518522  0.481478    0.0
 0.496341  0.503659    0.0
 0.344541  0.655459    1.0

In [44]:
function get_confusion(M, y_true, thresh = 0.5)
    d = Dict([:fp => 0, :fn => 0, :tn => 0, :tp => 0])
    n = size(M,1)
    for i=1:n 
        if M[i,2] > thresh
            if y_true[i] == 1.0
                d[:tp] += 1
            else
                d[:fp] += 1
            end
        else
            if y_true[i] == 1.0
                d[:fn] += 1
            else
                d[:tn] += 1
            end
        end
    end
    return d
end

get_confusion (generic function with 2 methods)

In [46]:
get_confusion(M, y_test, 0.5)

Dict{Symbol, Int64} with 4 entries:
  :tn => 196062
  :fp => 9974
  :tp => 57827
  :fn => 5068

In [55]:
measures("FScore")
m = MulticlassFScore()
m(mode.(p), y_test)

0.9239826633500551

In [59]:
predict(mach, X_test)[1:3]
e = evaluate!(mach, resampling=CV(nfolds=6), measures=[m, BalancedAccuracy(adjusted=true)])

Evaluating over 6 folds: 100%[=========================] Time: 0:00:52


PerformanceEvaluation object with these fields:
  model, measure, operation,
  measurement, per_fold, per_observation,
  fitted_params_per_fold, report_per_fold,
  train_test_rows, resampling, repeats
Extract:
┌───┬──────────────────────────────┬──────────────┬─────────────┐
│   │ measure                      │ operation    │ measurement │
├───┼──────────────────────────────┼──────────────┼─────────────┤
│ A │ MulticlassFScore(            │ predict_mode │ 0.931       │
│   │   beta = 1.0,                │              │             │
│   │   average = MacroAvg(),      │              │             │
│   │   return_type = LittleDict,  │              │             │
│   │   levels = nothing,          │              │             │
│   │   perm = nothing,            │              │             │
│   │   rev = nothing,             │              │             │
│   │   checks = true)             │              │             │
│ B │ BalancedAccuracy(            │ predict_mode │ 0.884       

In [61]:
e.measurement

2-element Vector{Float64}:
 0.9313422991300225
 0.8840891990606369

In [ ]:
#r = range(tree, :threshold, lower=0.1, upper=0.9)
tuned_tree = TunedModel(
    models=[tree],
    tuning=Explicit(),#RandomSearch(),
    resampling=CV(nfolds=6),
    range = nothing,#r,
    measure=m,
    n=30
)
mach2 = machine(tuned_tree, X, y) |> fit!
optimized_tree = report(mach2).best_model
optimized_tree.threshold

ArgumentError: ArgumentError: You need to specify `range=...`, unless `tuning=Explicit()` and and `models=...` is specified instead. 

In [63]:
tree

RandomForestClassifier(
  n_trees = 30, 
  max_depth = 6, 
  min_gain = 0.0, 
  min_records = 2, 
  max_features = 0, 
  splitting_criterion = BetaML.Utils.gini, 
  β = 0.0, 
  rng = Random._GLOBAL_RNG())